## Training Neural Networks

**Contents**
1. PyTorch Tensors
   1. Calculating PyTorch Gradients
2. Building a Simple Neural Network
3. Getting our Data
4. Setting up our training loop
5. Training our model

### Setup/Installations

In [ ]:
!pip install numpy torch

### PyTorch

we use PyTorch to build neural networks because it has support for gradient calculation

In [10]:
# creating a tensor in PyTorch
# Tensor: can be a vector, matrix, etc. but can have more dimensions (3, 4, ...)
import torch

tensor = torch.tensor([[1.2, 1.5], [1.6, 1.7]], requires_grad=True)

In [11]:
# seeing the tensor's contents: a 2x2 matrix
tensor

tensor([[1.2000, 1.5000],
        [1.6000, 1.7000]], requires_grad=True)

In [12]:
x = torch.tensor([[0.5, 0.7]], requires_grad=True)
x

tensor([[0.5000, 0.7000]], requires_grad=True)

In [13]:
# let's do a dot product!
y = x @ tensor
y

tensor([[1.7200, 1.9400]], grad_fn=<MmBackward0>)

In [17]:
# Let's sum the values in the y vector
output = torch.sum(y)
output

tensor(3.6600, grad_fn=<SumBackward0>)

In [18]:
# let's run a backward pass on output, this computes the gradients
output.backward()

In [21]:
# now we can access the gradients of all the computations we did
print(x.grad)
print(tensor.grad)

tensor([[2.7000, 3.3000]])
tensor([[0.5000, 0.5000],
        [0.7000, 0.7000]])


With these automatically generated gradients by a backward pass, we can use them to shift the weights to optimize for a loss function

However, we won't have to access gradients individually like we just did, PyTorch handles a lot of the low-level stuff for us

## Building a Simple Neural Network

We have built a neural network in NumPy before, but PyTorch has prebuilt primitives to make the process much easier, some important things are:
- `torch.nn.Module` - a class we can extend/inherit to create new neural networks
- `torch.nn.Linear` - the equivalent of a linear transformation by a weight and a bias, a simple neural network layer
- `torch.nn.functional.relu` - the relu activation function
- `torch.nn.functional.sigmoid` - the sigmoid activation function

PyTorch handles these low-level computations and allows for robust configuration. It will also automatically compute gradients for all of the operations

In [41]:
from torch import nn, Tensor

class NeuralNetwork(nn.Module): # inheriting from the base PyTorch module class
    def __init__(self, d_in: int, d_hidden: int, d_out: int) -> None: # the initialization function, which takes 3 parameters
        super().__init__() # calls the base constructor for initial setup

        self.linear1 = nn.Linear(in_features=d_in, out_features=d_hidden) # takes an input vector of size d_in, outputs a vector of size d_hidden
        self.linear2 = nn.Linear(in_features=d_hidden, out_features=d_out) # can take input from the previous linear layer to chain together

    def forward(self, x: Tensor) -> Tensor: # this is the method that tells the module what to do with the input
        # NOTE: the size of x must be d_in
        h = self.linear1(x) # first linear projection
        h = nn.functional.relu(h) # relu activation
        out = self.linear2(h) # second linear projection
        out = nn.functional.relu(out)

        return out

In [51]:
# testing out our model

neuralnet = NeuralNetwork(d_in=3, d_hidden=5, d_out=2)

In [52]:
# let's see the weights of our model
neuralnet.linear1.weight

Parameter containing:
tensor([[-0.4919, -0.0019, -0.4017],
        [-0.3067,  0.2290, -0.3321],
        [ 0.4997,  0.4455, -0.4952],
        [ 0.3109,  0.1463,  0.5115],
        [-0.1808,  0.4914,  0.3718]], requires_grad=True)

In [53]:
# let's see the biases in the second layer of our model
neuralnet.linear2.bias

Parameter containing:
tensor([-0.3553, -0.1153], requires_grad=True)

In [54]:
# let's run a sample input
x = torch.rand(3)
x

tensor([0.3434, 0.8246, 0.3863])

In [55]:
neuralnet(x)

tensor([0.0000, 0.1048], grad_fn=<ReluBackward0>)

### Getting Data to Train our Model

In [59]:
from sklearn.datasets import load_wine

data_X, data_y = load_wine(return_X_y=True, as_frame=True)
data_X.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [60]:
# seeing our dataset's length
len(data_X)

178

In [64]:
# distribution of classes
data_y.value_counts()

target
1    71
0    59
2    48
Name: count, dtype: int64

While SKLearn datasets work well for working with sklearn models, PyTorch datasets (`torch.utils.data.Dataset`) have many utilities that are helpful for training neural networks. Luckily, it is pretty easy to convert between the two

To create a PyTorch dataset, we must create a class that inherits from the base class, such as `class WineDataset(torch.utils.data.Dataset):`

We must then implement the following methods:
- `__init__()` - this initializes the data
- `__len__()` - this returns the length of the data
- `__getitem__(self, index)` - this returns an item at a specific index given as input

In [65]:
from torch.utils.data import Dataset

class WineData(Dataset):
    def __init__(self, data_X, data_y):
        super().__init__() # initial setup by PyTorch

        self.x = data_X
        self.y = data_y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index: int):
        return self.x[index], self.y[index]

After creating these datasets, PyTorch allows us to create DataLoaders (`torch.utils.data.DataLoader`), which can help us retrieve information sequentially or in groups to feed to our model during training

### The Training Loop

While previous methods usually have their training loop under the hood, it's common to write a custom one in PyTorch. We will write a simple loop that consists of looping through the following steps per item in the dataset:

1. Retrieve the input features and expected output from our dataloader
2. Pass the inputs through the model
3. Calculate the loss function given the model's output and the expected output
4. Calculate the **gradients** based on the loss function
5. *Step* the gradients in the direction that minimizes the loss function
   1. We have optimizers that can do this, which PyTorch provides

Some initial setup we'll have to do:
- Setting up our loss function
- Setting up our optimizer
- Setting up storage to hold losses over time

A little bit of terminology:
- Epoch - one full iteration through the dataset, small datasets usually have large numbers of epochs
- Batch - a set of data passed to the model at once. Instead of passing one data item each time, passing multiple allows the model to compute them in parallel
  - Batch sizes are usually powers of 2 to conform to hardware constraints
- Learning rate - determines what percentage of the gradients we alter the weights by
  - e.g. if gradient for a parameter is 10 and the learning rate is 0.01, the model will shift that weight by 0.1
  - Learning rates must be chosen to balance speed but prevent instability

In [ ]:
# a function that we can use to organize our data into batches for our model

def collate(x: list[tuple[Tensor, Tensor]]):
    return torch.stack([i[0] for i in x]), torch.stack([i[1] for i in x])

In [66]:
# the training loop!
from torch.utils.data import DataLoader
from tqdm import tqdm

# some config
NUMBER_OF_EPOCHS = 10
BATCH_SIZE = 4
LEARNING_RATE = 0.01

def train(model: NeuralNetwork, data: WineData):
    # setting up our loss function: this is a multiclass classification problem, we'll use cross entropy
    loss_function = nn.CrossEntropyLoss()

    losses = [] # a list to store our losses over time

    # setting up our optimizer, we'll use stochastic gradient descent for this
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(NUMBER_OF_EPOCHS): # for each iteration in the dataset
        # we'll set up our data for each epoch

        loader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate) # shuffling prevents the model from learning the dataset's order over time

        for index, data_item in tqdm(enumerate(loader), desc=f"Training our model on epoch {epoch}:"):
            optimizer.zero_grad() # resets our optimizer after the last gradient step
            input_data, target = data_item # getting the inputs and outputs from the batch

            # compute the outputs of the neural network
            output = model(input_data)

            #compute the loss
            loss = loss_function(output, target) # compares the predicted and expected outputs
            loss.backward() # computes gradients automatically

            losses.append(loss.item()) # adding our loss to the output

            optimizer.step() # alter the weights based on the gradients

    return model, losses